In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [2]:
df = pd.read_csv('/content/drive/MyDrive/machine learning projects/training set/fake news train.csv')
test = pd.read_csv('/content/drive/MyDrive/machine learning projects/training set/fake news test.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df=df.fillna('')
test=test.fillna('')

In [6]:
df['total']=df['author']+'-'+df['title']
test['total']=df['author']+'-'+df['title']

In [10]:
X=df.drop('label',axis=1)
Y=df['label']
print(X.shape)
print(Y.shape)

(20800, 5)
(20800,)


In [12]:
voc_size=5000
msg=X.copy()
msg_test=test.copy()
# 가지치기를 위한 복사

In [13]:
#Downloading stopwords 
#Stopwords are the words in any language which does not add much meaning to a sentence.
#They can safely be ignored without sacrificing the meaning of the sentence.
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
#We will be using Stemming here
#Stemming map words to their root forms
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [15]:
#Applying stemming and some preprocessing
for i in range(len(msg)):
  review = re.sub('[^a-zA-Z]',' ',msg['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [16]:
#Applying stemming and some preprocessing for test data
corpus_test = []
for i in range(len(msg_test)):
  review = re.sub('[^a-zA-Z]',' ',msg_test['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [17]:
# Converting to one hot representation
onehot_rep = [one_hot(words,voc_size)for words in corpus]
onehot_rep_test = [one_hot(words,voc_size)for words in corpus_test]

In [23]:
#Padding Sentences to make them of same size
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

Creating and training model

In [24]:
#We have used embedding layers with LSTM
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 40)            200000    
                                                                 
 dropout_3 (Dropout)         (None, 25, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                6464      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                

In [26]:
#Converting into numpy array
X_final = np.array(embedded_docs)
Y_final = np.array(Y)
test_final = np.array(embedded_docs_test)
X_final.shape,Y_final.shape,test_final.shape

((20800, 25), (20800,), (5200, 25))

In [28]:
#training model
model.fit(X_final,Y_final,epochs=20,batch_size=64)

Epoch 1/20
325/325 [==============================] - 14s 42ms/step - loss: 3.4340e-04 - accuracy: 0.9999
Epoch 2/20
325/325 [==============================] - 11s 33ms/step - loss: 0.0024 - accuracy: 0.9998
Epoch 3/20
325/325 [==============================] - 11s 32ms/step - loss: 0.0021 - accuracy: 0.9995
Epoch 4/20
325/325 [==============================] - 12s 36ms/step - loss: 4.3618e-04 - accuracy: 0.9998
Epoch 5/20
325/325 [==============================] - 10s 31ms/step - loss: 0.0013 - accuracy: 0.9997
Epoch 6/20
325/325 [==============================] - 10s 30ms/step - loss: 7.8476e-05 - accuracy: 1.0000
Epoch 7/20
325/325 [==============================] - 10s 30ms/step - loss: 4.7281e-04 - accuracy: 0.9999
Epoch 8/20
325/325 [==============================] - 10s 30ms/step - loss: 6.8365e-04 - accuracy: 0.9998
Epoch 9/20
325/325 [==============================] - 10s 30ms/step - loss: 2.1015e-04 - accuracy: 0.9999
Epoch 10/20
325/325 [==============================] - 10s

Creating Submission file

In [38]:
y_pred = (model.predict(test_final) > 0.5).astype("int32")

In [39]:
final_sub = pd.DataFrame()
final_sub['id']=test['id']
final_sub['label'] = y_pred
final_sub.to_csv('final_sub.csv',index=False)

In [40]:
final_sub.head()

,id,label
0,20800,1
1,20801,0
2,20802,1
3,20803,1
4,20804,1
